In [22]:
from pathlib import Path
import numpy as np
import pandas as pd

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 140)

SEED = 42
np.random.seed(SEED)

In [23]:
RAIZ_PROYECTO = Path.cwd()  # porque ya confirmaste que estás en Mantenimiento Industrial
ruta_parquet = RAIZ_PROYECTO / "data" / "processed" / "azure_pm" / "dataset_survival_diario.parquet"

print("Ruta parquet:", ruta_parquet)
print("Existe:", ruta_parquet.exists())

# Debug: listar archivos en la carpeta por si el nombre cambia
carpeta = ruta_parquet.parent
print("\nContenido carpeta:", carpeta)
if carpeta.exists():
    for p in carpeta.glob("*"):
        print("-", p.name)

dataset = pd.read_parquet(ruta_parquet)

print("\n✅ Cargado:", dataset.shape)
dataset.head()


Ruta parquet: c:\Users\sebas\OneDrive\Desktop\Proyecto Chatbot\Mantenimiento Industrial\data\processed\azure_pm\dataset_survival_diario.parquet
Existe: True

Contenido carpeta: c:\Users\sebas\OneDrive\Desktop\Proyecto Chatbot\Mantenimiento Industrial\data\processed\azure_pm
- dataset_modelo.parquet
- dataset_survival_diario.parquet

✅ Cargado: (36600, 28)


,machineID,fecha,datetime_mean,datetime_std,datetime_min,datetime_max,volt_mean,volt_std,volt_min,volt_max,rotate_mean,rotate_std,rotate_min,rotate_max,pressure_mean,pressure_std,pressure_min,pressure_max,vibration_mean,vibration_std,vibration_min,vibration_max,evento,model,age,proxima_falla,dias_hasta_falla,censurado
0,1,2015-01-01,2015-01-01 14:30:00,0 days 05:20:18.740853656,2015-01-01 06:00:00,2015-01-01 23:00:00,167.576533,9.300337,151.335682,182.739113,440.515328,49.590263,346.149335,527.349825,98.522345,10.588562,75.237905,113.077935,40.049623,5.739395,25.990511,51.021486,0,model3,18,2015-01-05,4,0
1,1,2015-01-02,2015-01-02 11:30:00,0 days 07:04:15.844122715,2015-01-02 00:00:00,2015-01-02 23:00:00,169.795758,15.742155,140.776309,200.872430,446.832666,38.800266,384.645962,519.452812,98.454608,11.679314,78.880780,127.014498,39.271645,5.579524,29.527665,52.355876,0,model3,18,2015-01-05,3,0
2,1,2015-01-03,2015-01-03 11:30:00,0 days 07:04:15.844122715,2015-01-03 00:00:00,2015-01-03 23:00:00,171.862244,11.182853,154.199258,194.942847,459.204742,47.387959,374.127148,568.972310,97.998233,8.884765,85.246610,116.008404,48.074091,8.194927,32.323616,66.764515,0,model3,18,2015-01-05,2,0
3,1,2015-01-04,2015-01-04 11:30:00,0 days 07:04:15.844122715,2015-01-04 00:00:00,2015-01-04 23:00:00,174.792428,19.224657,129.016707,215.656488,448.743201,34.008026,365.213804,517.348533,101.452266,10.807630,82.400818,118.853452,52.190268,5.081258,41.674887,62.464103,0,model3,18,2015-01-05,1,0
4,1,2015-01-05,2015-01-05 11:30:00,0 days 07:04:15.844122715,2015-01-05 00:00:00,2015-01-05 23:00:00,171.018408,17.900560,127.163620,202.520488,454.822750,47.803621,376.719605,575.505189,102.363114,10.672868,78.721961,126.464580,43.330311,8.087134,33.156011,59.577251,1,model3,18,2015-01-05,0,0


In [24]:
dataset = dataset.sort_values(["machineID", "fecha"]).reset_index(drop=True)

print("Filas:", len(dataset))
print("Máquinas:", dataset["machineID"].nunique())
print("Rango fechas:", dataset["fecha"].min(), "→", dataset["fecha"].max())

dataset[["machineID","fecha","evento","dias_hasta_falla","censurado"]].head()

Filas: 36600
Máquinas: 100
Rango fechas: 2015-01-01 00:00:00 → 2016-01-01 00:00:00


,machineID,fecha,evento,dias_hasta_falla,censurado
0,1,2015-01-01,0,4,0
1,1,2015-01-02,0,3,0
2,1,2015-01-03,0,2,0
3,1,2015-01-04,0,1,0
4,1,2015-01-05,1,0,0


In [25]:
columnas_stats = [c for c in dataset.columns if c.endswith(("_mean", "_std", "_min", "_max"))]
print("N columnas stats:", len(columnas_stats))
print("Ejemplos:", columnas_stats[:12])


N columnas stats: 20
Ejemplos: ['datetime_mean', 'datetime_std', 'datetime_min', 'datetime_max', 'volt_mean', 'volt_std', 'volt_min', 'volt_max', 'rotate_mean', 'rotate_std', 'rotate_min', 'rotate_max']


In [26]:
columnas_mean = [c for c in columnas_stats if c.endswith("_mean")]
print("N columnas mean:", len(columnas_mean))
print("Ejemplos:", columnas_mean[:12])


N columnas mean: 5
Ejemplos: ['datetime_mean', 'volt_mean', 'rotate_mean', 'pressure_mean', 'vibration_mean']


In [27]:
# columnas que terminan en _mean
columnas_mean = [c for c in columnas_stats if c.endswith("_mean")]

# quedarnos SOLO con las que son numéricas (excluir datetime/objetos)
columnas_mean_numericas = [c for c in columnas_mean if pd.api.types.is_numeric_dtype(dataset[c])]

# Debug: mostrar las que quedaron fuera (para confirmar causa)
columnas_excluidas = [c for c in columnas_mean if c not in columnas_mean_numericas]
print("N columnas mean (total):", len(columnas_mean))
print("N columnas mean numéricas:", len(columnas_mean_numericas))
print("Excluidas por no numéricas:", columnas_excluidas[:20])

N columnas mean (total): 5
N columnas mean numéricas: 4
Excluidas por no numéricas: ['datetime_mean']


In [28]:
LAGS = [1, 2, 3, 7]

for lag in LAGS:
    for col in columnas_mean_numericas:
        dataset[f"{col}_lag{lag}d"] = dataset.groupby("machineID")[col].shift(lag)

print("✅ Lags creados. Columnas totales:", dataset.shape[1])

✅ Lags creados. Columnas totales: 44


In [29]:
VENTANAS = [3, 7, 14]

for ventana in VENTANAS:
    for col in columnas_mean_numericas:
        serie = dataset.groupby("machineID")[col]
        dataset[f"{col}_roll{ventana}d_mean"] = serie.transform(lambda s: s.rolling(ventana, min_periods=1).mean())
        dataset[f"{col}_roll{ventana}d_std"]  = serie.transform(lambda s: s.rolling(ventana, min_periods=1).std())

print("✅ Rolling creado. Columnas totales:", dataset.shape[1])

✅ Rolling creado. Columnas totales: 68


In [30]:
for col in columnas_mean_numericas:
    dataset[f"{col}_tendencia_7d"] = dataset[col] - dataset.groupby("machineID")[col].shift(7)

print("✅ Tendencias creadas. Columnas totales:", dataset.shape[1])

✅ Tendencias creadas. Columnas totales: 72


# 📊 Notebook 02 — Ingeniería de Features Temporales

## Estado actual del pipeline

En este punto del proyecto ya se ha construido una base sólida de variables explicativas a nivel `machineID`–`fecha`, integrando información histórica y tendencias de sensores.

Hasta ahora, el flujo implementado incluye:

---

### ✅ 1. Features diarios base
- Variables agregadas desde telemetría:
  - Media, desviación estándar, mínimo y máximo por sensor.
- Provenientes del dataset generado en el Notebook 01.
- Representan el estado operativo diario de cada máquina.

---

### ✅ 2. Lags (memoria temporal)
- Se generaron rezagos temporales para sensores numéricos (`*_mean`):
  - `lag1d`, `lag2d`, `lag3d`, `lag7d`.
- Objetivo:
  - Permitir que el modelo incorpore información del pasado reciente.
  - Capturar patrones previos a una falla.

---

### ✅ 3. Ventanas móviles (rolling features)
- Se calcularon ventanas móviles de 3, 7 y 14 días:
  - `rollXd_mean` → promedio móvil.
  - `rollXd_std` → volatilidad local.
- Objetivo:
  - Capturar estabilidad vs. inestabilidad operativa.
  - Detectar procesos de degradación progresiva.

---

### ✅ 4. Variables de tendencia
- Se construyó una tendencia simple a 7 días:
  - `tendencia_7d = valor_actual − valor_hace_7_días`.
- Objetivo:
  - Identificar direccionalidad en el comportamiento de sensores.
  - Señales tempranas de deterioro.

> Nota técnica:  
> Todas las operaciones temporales se aplicaron exclusivamente sobre columnas numéricas, evitando errores con variables de tipo `datetime`.

---

## Próximos pasos

A continuación, el pipeline continúa con las etapas necesarias para dejar el dataset listo para entrenamiento:

---

### 🔜 5. Definición de variables objetivo (targets)
Se crearán etiquetas por horizonte temporal:

- `falla_7d`
- `falla_14d`
- `falla_30d`

Estas variables indican si una máquina fallará dentro del horizonte especificado.

---

### 🔜 6. Construcción del dataset para modelado
Se generará `dataset_modelo` aplicando:

- Filtrado inicial:
  - Exclusión de observaciones censuradas (`dias_hasta_falla = -1`).
- Prevención de fuga de información (leakage):
  - Eliminación de variables futuras (`proxima_falla`, `dias_hasta_falla`, etc.).

---

### 🔜 7. Control de calidad post-feature engineering
Se evaluará:

- Porcentaje de valores nulos generados por lags y ventanas.
- Distribución de nulos por columna y por fila.

Esta información guiará la estrategia de imputación en el Notebook 03.

---

### 🔜 8. Persistencia del dataset final
El dataset listo para entrenamiento se almacenará como:


In [31]:
def crear_objetivo_horizonte(df, horizonte):
    return (
        (df["dias_hasta_falla"] >= 0) &
        (df["dias_hasta_falla"] <= horizonte)
    ).astype(int)


dataset["falla_7d"] = crear_objetivo_horizonte(dataset, 7)
dataset["falla_14d"] = crear_objetivo_horizonte(dataset, 14)
dataset["falla_30d"] = crear_objetivo_horizonte(dataset, 30)


# Métricas solo en no censurados
no_censurados = dataset["dias_hasta_falla"] != -1

rate_7d  = round(dataset.loc[no_censurados, "falla_7d"].mean() * 100, 2)
rate_14d = round(dataset.loc[no_censurados, "falla_14d"].mean() * 100, 2)
rate_30d = round(dataset.loc[no_censurados, "falla_30d"].mean() * 100, 2)

print(f"Rate falla 7d : {rate_7d}%")
print(f"Rate falla 14d: {rate_14d}%")
print(f"Rate falla 30d: {rate_30d}%")


Rate falla 7d : 17.72%
Rate falla 14d: 32.81%
Rate falla 30d: 56.94%


In [32]:
# Filtrar observaciones con target válido
dataset_modelo = dataset[dataset["dias_hasta_falla"] != -1].copy()


# Columnas que generan fuga de información
columnas_excluir = [
    "evento",
    "proxima_falla",
    "dias_hasta_falla",
    "censurado"
]

if "datetime" in dataset_modelo.columns:
    columnas_excluir.append("datetime")


dataset_modelo = dataset_modelo.drop(
    columns=[c for c in columnas_excluir if c in dataset_modelo.columns],
    errors="ignore"
)


print("Dataset modelo:")
print("Filas:", len(dataset_modelo))
print("Columnas:", dataset_modelo.shape[1])

dataset_modelo.head()


Dataset modelo:
Filas: 31567
Columnas: 71


,machineID,fecha,datetime_mean,datetime_std,datetime_min,datetime_max,volt_mean,volt_std,volt_min,volt_max,rotate_mean,rotate_std,rotate_min,rotate_max,pressure_mean,pressure_std,pressure_min,pressure_max,vibration_mean,vibration_std,vibration_min,vibration_max,model,age,volt_mean_lag1d,rotate_mean_lag1d,pressure_mean_lag1d,vibration_mean_lag1d,volt_mean_lag2d,rotate_mean_lag2d,pressure_mean_lag2d,vibration_mean_lag2d,volt_mean_lag3d,rotate_mean_lag3d,pressure_mean_lag3d,vibration_mean_lag3d,volt_mean_lag7d,rotate_mean_lag7d,pressure_mean_lag7d,vibration_mean_lag7d,volt_mean_roll3d_mean,volt_mean_roll3d_std,rotate_mean_roll3d_mean,rotate_mean_roll3d_std,pressure_mean_roll3d_mean,pressure_mean_roll3d_std,vibration_mean_roll3d_mean,vibration_mean_roll3d_std,volt_mean_roll7d_mean,volt_mean_roll7d_std,rotate_mean_roll7d_mean,rotate_mean_roll7d_std,pressure_mean_roll7d_mean,pressure_mean_roll7d_std,vibration_mean_roll7d_mean,vibration_mean_roll7d_std,volt_mean_roll14d_mean,volt_mean_roll14d_std,rotate_mean_roll14d_mean,rotate_mean_roll14d_std,pressure_mean_roll14d_mean,pressure_mean_roll14d_std,vibration_mean_roll14d_mean,vibration_mean_roll14d_std,volt_mean_tendencia_7d,rotate_mean_tendencia_7d,pressure_mean_tendencia_7d,vibration_mean_tendencia_7d,falla_7d,falla_14d,falla_30d
0,1,2015-01-01,2015-01-01 14:30:00,0 days 05:20:18.740853656,2015-01-01 06:00:00,2015-01-01 23:00:00,167.576533,9.300337,151.335682,182.739113,440.515328,49.590263,346.149335,527.349825,98.522345,10.588562,75.237905,113.077935,40.049623,5.739395,25.990511,51.021486,model3,18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,167.576533,NaN,440.515328,NaN,98.522345,NaN,40.049623,NaN,167.576533,NaN,440.515328,NaN,98.522345,NaN,40.049623,NaN,167.576533,NaN,440.515328,NaN,98.522345,NaN,40.049623,NaN,NaN,NaN,NaN,NaN,1,1,1
1,1,2015-01-02,2015-01-02 11:30:00,0 days 07:04:15.844122715,2015-01-02 00:00:00,2015-01-02 23:00:00,169.795758,15.742155,140.776309,200.872430,446.832666,38.800266,384.645962,519.452812,98.454608,11.679314,78.880780,127.014498,39.271645,5.579524,29.527665,52.355876,model3,18,167.576533,440.515328,98.522345,40.049623,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,168.686146,1.569229,443.673997,4.467033,98.488477,0.047897,39.660634,0.550113,168.686146,1.569229,443.673997,4.467033,98.488477,0.047897,39.660634,0.550113,168.686146,1.569229,443.673997,4.467033,98.488477,0.047897,39.660634,0.550113,NaN,NaN,NaN,NaN,1,1,1
2,1,2015-01-03,2015-01-03 11:30:00,0 days 07:04:15.844122715,2015-01-03 00:00:00,2015-01-03 23:00:00,171.862244,11.182853,154.199258,194.942847,459.204742,47.387959,374.127148,568.972310,97.998233,8.884765,85.246610,116.008404,48.074091,8.194927,32.323616,66.764515,model3,18,169.795758,446.832666,98.454608,39.271645,167.576533,440.515328,98.522345,40.049623,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,169.744845,2.143309,448.850912,9.506762,98.325062,0.285061,42.465120,4.873062,169.744845,2.143309,448.850912,9.506762,98.325062,0.285061,42.465120,4.873062,169.744845,2.143309,448.850912,9.506762,98.325062,0.285061,42.465120,4.873062,NaN,NaN,NaN,NaN,1,1,1
3,1,2015-01-04,2015-01-04 11:30:00,0 days 07:04:15.844122715,2015-01-04 00:00:00,2015-01-04 23:00:00,174.792428,19.224657,129.016707,215.656488,448.743201,34.008026,365.213804,517.348533,101.452266,10.807630,82.400818,118.853452,52.190268,5.081258,41.674887,62.464103,model3,18,171.862244,459.204742,97.998233,48.074091,169.795758,446.832666,98.454608,39.271645,167.576533,440.515328,98.522345,40.049623,NaN,NaN,NaN,NaN,172.150143,2.510745,451.593536,6.660358,99.301702,1.876370,46.512001,6.599454,171.006741,3.071163,448.823984,7.762426,99.106863,1.580830,44.896407,6.282976,171.006741,3.071163,448.823984,7.762426,99.106863,1.580830,44.896407,6.282976,NaN,NaN,NaN,NaN,1,1,1
4,1,2015-01-05,2015-01-05 11:30:00,0 days 07:04:15.844122715,2015-01-05 00:00:00,2015-01-05 23:00:00,171.018408,17.900560,127.163620,202.520488,454.822750,47.803621,376.719605,575.505189,102.363114,10.672868,78.721961,126.4

In [33]:
porcentaje_nulos = (
    dataset_modelo
    .isna()
    .mean()
    .mul(100)
    .round(2)
    .sort_values(ascending=False)
)

print("Top 15 columnas con más nulos (%):")
print(porcentaje_nulos.head(15))


# Filas con al menos un nulo
porcentaje_filas_nulos = round(
    dataset_modelo.isna().any(axis=1).mean() * 100,
    2
)

print(f"\n% filas con al menos un nulo: {porcentaje_filas_nulos}%")


Top 15 columnas con más nulos (%):
rotate_mean_tendencia_7d       2.17
pressure_mean_tendencia_7d     2.17
vibration_mean_tendencia_7d    2.17
volt_mean_tendencia_7d         2.17
pressure_mean_lag7d            2.17
rotate_mean_lag7d              2.17
volt_mean_lag7d                2.17
vibration_mean_lag7d           2.17
vibration_mean_lag3d           0.93
pressure_mean_lag3d            0.93
rotate_mean_lag3d              0.93
volt_mean_lag3d                0.93
volt_mean_lag2d                0.62
pressure_mean_lag2d            0.62
vibration_mean_lag2d           0.62
dtype: float64

% filas con al menos un nulo: 2.17%


In [34]:
dataset_modelo = (
    dataset_modelo
    .sort_values(["machineID", "fecha"])
    .groupby("machineID")
    .apply(lambda df: df.ffill())
    .reset_index(drop=True)
)


# Revisar nulos restantes
nulos_restantes = round(
    dataset_modelo.isna().mean().max() * 100,
    2
)

print(f"Máx % nulos tras imputación: {nulos_restantes}%")


Máx % nulos tras imputación: 2.17%


C:\Users\sebas\AppData\Local\Temp\ipykernel_11276\2309698403.py:5: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda df: df.ffill())


In [35]:
from sklearn.preprocessing import StandardScaler

# Excluir columnas de identificación y targets
columnas_no_features = {"machineID", "fecha", "falla_7d", "falla_14d", "falla_30d"}

# Candidatas
columnas_candidatas = [c for c in dataset_modelo.columns if c not in columnas_no_features]

# Separar numéricas vs no numéricas
columnas_numericas = [c for c in columnas_candidatas if pd.api.types.is_numeric_dtype(dataset_modelo[c])]
columnas_no_numericas = [c for c in columnas_candidatas if c not in columnas_numericas]

print("Features numéricas:", len(columnas_numericas))
print("No numéricas (no se escalan):", len(columnas_no_numericas))
print("Ejemplos no numéricas:", columnas_no_numericas[:10])

# Escalado
escalador = StandardScaler()
dataset_modelo[columnas_numericas] = escalador.fit_transform(dataset_modelo[columnas_numericas])

print("✅ Escalado aplicado a numéricas.")


Features numéricas: 61
No numéricas (no se escalan): 5
Ejemplos no numéricas: ['datetime_mean', 'datetime_std', 'datetime_min', 'datetime_max', 'model']


✅ Escalado aplicado a numéricas.


In [36]:
DIRECTORIO_SALIDA = Path.cwd() / "data" / "processed" / "azure_pm"
DIRECTORIO_SALIDA.mkdir(parents=True, exist_ok=True)

ruta_salida = DIRECTORIO_SALIDA / "dataset_modelo.parquet"

dataset_modelo.to_parquet(ruta_salida, index=False)

print("✅ Guardado en:")
print(ruta_salida.resolve())

✅ Guardado en:
C:\Users\sebas\OneDrive\Desktop\Proyecto Chatbot\Mantenimiento Industrial\data\processed\azure_pm\dataset_modelo.parquet


In [37]:
filas, columnas = dataset_modelo.shape

print(f"Filas: {filas:,}")
print(f"Columnas: {columnas:,}")

Filas: 31,567
Columnas: 71


In [38]:
n_maquinas = dataset_modelo["machineID"].nunique()

print(f"Máquinas únicas: {n_maquinas}")

Máquinas únicas: 98


In [39]:
porc_filas_nulos = round(
    dataset_modelo.isna().any(axis=1).mean() * 100,
    2
)

print(f"% filas con al menos un nulo: {porc_filas_nulos}%")


% filas con al menos un nulo: 2.17%


In [40]:
rate_7d = round(dataset_modelo["falla_7d"].mean() * 100, 2)
rate_14d = round(dataset_modelo["falla_14d"].mean() * 100, 2)
rate_30d = round(dataset_modelo["falla_30d"].mean() * 100, 2)

print(f"Rate falla ≤7d : {rate_7d}%")
print(f"Rate falla ≤14d: {rate_14d}%")
print(f"Rate falla ≤30d: {rate_30d}%")


Rate falla ≤7d : 17.72%
Rate falla ≤14d: 32.81%
Rate falla ≤30d: 56.94%


In [41]:
porc_censurados = round(
    (dataset["dias_hasta_falla"] == -1).mean() * 100,
    2
)

print(f"% observaciones censuradas: {porc_censurados}%")


% observaciones censuradas: 13.75%


In [42]:
resumen = {
    "filas": filas,
    "columnas": columnas,
    "maquinas": n_maquinas,
    "pct_filas_nulos": porc_filas_nulos,
    "rate_7d": rate_7d,
    "rate_14d": rate_14d,
    "rate_30d": rate_30d,
}

resumen


{'filas': 31567,
 'columnas': 71,
 'maquinas': 98,
 'pct_filas_nulos': np.float64(2.17),
 'rate_7d': np.float64(17.72),
 'rate_14d': np.float64(32.81),
 'rate_30d': np.float64(56.94)}

# Conclusiones — Notebook 02

En este notebook se construyó el dataset final para modelado,
integrando información histórica, temporal y de degradación
a nivel máquina–día.

A partir del dataset survival diario, se desarrolló un conjunto
robusto de variables explicativas mediante:

- Lags temporales (1, 2, 3 y 7 días).
- Ventanas móviles de 3, 7 y 14 días.
- Métricas de tendencia a 7 días.
- Estadísticos diarios de sensores.

El proceso resultó en un dataset con:

- **31.567 observaciones**.
- **71 variables**.
- **98 máquinas monitoreadas**.

Tras la imputación temporal por máquina, el porcentaje de filas
con valores faltantes se redujo a **2,17%**, lo que indica un
alto nivel de completitud y consistencia de la información.

Las tasas de falla por horizonte muestran una progresión clara
y consistente:

- **Falla ≤ 7 días:** 17,72%
- **Falla ≤ 14 días:** 32,81%
- **Falla ≤ 30 días:** 56,94%

Estos resultados confirman la existencia de señal predictiva
explotable, especialmente en horizontes medios y largos,
lo que habilita el uso de modelos probabilísticos para
planificación anticipada de mantenimiento.

El dataset generado (`dataset_modelo.parquet`) queda preparado
para ser utilizado en el Notebook 03, donde se abordará el
entrenamiento, validación temporal, calibración de probabilidades
y evaluación comparativa de modelos.

Con esto se completa la fase de ingeniería de features del pipeline,
estableciendo una base sólida para la toma de decisiones
basada en riesgo y costo esperado.
